# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load and read csv

weather_load = 'citydataframe.csv'
vacay_df = pd.read_csv(weather_load, low_memory=False)
vacay_df.head()

,Unnamed: 0,City,Latitude,Temperature,Cloudiness,Humidity,Wind Speed,Longitude,Country
0,0,Rikitea,-23.12,75.65,52,81,12.62,-134.97,PF
1,1,Punta Arenas,-53.15,32.86,20,86,4.70,-70.92,CL
2,2,Ulaanbaatar,47.91,59.00,75,34,4.74,106.88,MN
3,3,Kurchum,48.57,63.82,78,52,8.70,83.65,KZ
4,4,Albany,42.60,69.44,25,51,1.25,-73.97,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacay_df[["Latitude", "Longitude"]]

weights = vacay_df["Humidity"]
# Plot Heatmap
fig = gmaps.figure()

#Layout format
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout) #
heat_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Drop values that dont match computations
w_df = vacay_df
w_df.drop(w_df[w_df["Temperature"] > 80].index ,inplace=True)   
w_df.drop(w_df[w_df["Temperature"] < 70].index ,inplace=True)
w_df.drop(w_df[w_df["Cloudiness"] != 0].index ,inplace=True)
w_df.drop(w_df[w_df["Wind Speed"] < 10].index ,inplace=True)

w_df.describe()

,Unnamed: 0,Latitude,Temperature,Cloudiness,Humidity,Wind Speed,Longitude
count,7.000000,7.000000,7.000000,7.0,7.000000,7.000000,7.000000
mean,328.285714,17.228571,74.267143,0.0,47.857143,14.604286,4.484286
std,162.766412,25.607555,3.202851,0.0,23.433797,4.432309,65.665652
min,61.000000,-26.650000,70.450000,0.0,11.000000,10.020000,-114.060000
25%,244.000000,8.045000,71.860000,0.0,32.500000,10.805000,-13.990000
50%,382.000000,27.980000,73.400000,0.0,60.000000,12.930000,21.550000
75%,414.500000,30.190000,76.605000,0.0,65.000000,19.070000,29.215000
max,538.000000,42.800000,79.090000,0.0,69.000000,19.530000,93.450000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create column for hotels
hotel_df = w_df
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Latitude,Temperature,Cloudiness,Humidity,Wind Speed,Longitude,Country,Hotel Name
61,61,Marsá Maţrūḩ,31.35,71.60,0,68,12.93,27.25,EG,
168,168,Kumul,42.80,79.09,0,21,10.02,93.45,CN,
320,320,Guerrero Negro,27.98,70.45,0,62,19.53,-114.06,MX,
382,382,Jalu,29.03,72.12,0,69,18.68,21.55,LY,
411,411,Lüderitz,-26.65,73.40,0,11,19.46,15.16,NaN,
418,418,Barra,-11.09,76.21,0,60,10.20,-43.14,BR,
538,538,Assiut,27.18,77.00,0,44,11.41,31.18,EG,


In [39]:
# Import URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Loop through rows and populate hotel column
for index, row in hotel_df.iterrows():
    coordinates = str(hotel_df.loc[index,'Latitude']) +',' + str(hotel_df.loc[index,'Longitude'])
    params = {
        "key": g_key,
        "location": coordinates,
        "radius": 5000,
        "type": "lodging"
        }
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
#Check for errors and passover
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except(IndexError):
        continue
        
hotel_df.head()

,Unnamed: 0,City,Latitude,Temperature,Cloudiness,Humidity,Wind Speed,Longitude,Country,Hotel Name
61,61,Marsá Maţrūḩ,31.35,71.60,0,68,12.93,27.25,EG,Beau Site
168,168,Kumul,42.80,79.09,0,21,10.02,93.45,CN,
320,320,Guerrero Negro,27.98,70.45,0,62,19.53,-114.06,MX,"Casa Laguna, Bed & Breakfast"
382,382,Jalu,29.03,72.12,0,69,18.68,21.55,LY,Jalu Hotel
411,411,Lüderitz,-26.65,73.40,0,11,19.46,15.16,NaN,LÜDERITZ NEST HOTEL


In [41]:
# NOTE: Do not change any of the code in this cell
narrowed_city_df = hotel_df
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [49]:
# Add marker layer ontop of heat map
df_heat= pd.read_csv(weather_load)
locations = df_heat[["Latitude", "Longitude"]]
Humidity = df_heat["Humidity"]

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
# Display Map
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout) #
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)
fig.add_layer(heat_layer)

# markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))